In [17]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

In [18]:
###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv') 
input_data.shape

(84840, 50)

In [19]:
df = input_data.drop(columns=['frmDist'])
df = df.sort_values(by='date')

# 데이터를 훈련 세트와 테스트 세트로 분할
'''
데이터를 훈련 세트와 테스트 세트로 나누는 데 사용됩니다. 
이는 모델의 성능을 평가하기 위해 데이터를 분리하는 일반적인 절차입니다.
'''
# 데이터셋을 data, target으로 변수분리
X = df[df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [20]:
model = Sequential()

# Input layer
model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting

# Second LSTM layer
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))  # Dropout layer

# Third LSTM layer
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))  # Dropout layer

# Fully connected layers
model.add(Dense(25))
model.add(Dense(2))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary to show the modified architecture
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 47, 128)           66560     
                                                                 
 dropout (Dropout)           (None, 47, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 47, 128)           131584    
                                                                 
 dropout_1 (Dropout)         (None, 47, 128)           0         
                                                                 
 lstm_6 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 25)               

In [21]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
67872/67872 [==============================] - 1674s 25ms/step - loss: 156153118720.0000
Epoch 2/10
55990/67872 [=======================>......] - ETA: 4:45 - loss: 97807876096.0000

In [15]:
y_pred = model.predict(X_test)

531/531 [==============================] - 9s 16ms/step


In [16]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 350843.8724023318
R2_score: -0.05908874638970474
